# Rental Price Prediction using Linear Regression Method
This file is dedicated to predicting Rental prices of housing in Singapore.
This file would be using the Linear Regression Method to predicting the rental prices.

## Data Collection
- The following codes below in this section is dedicated to reading and combining csv files into 1 dataframe.
- We also import all necessary modules in this section as well.


In [8]:
# Necessary Imports
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import ensemble
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Will add more imports here later

In [9]:

#pre-set file path
file_path = "../dataset/rental-prediction/Non-Landed Housing/"

#combining all into one file for non-landed
df1 = pd.read_csv(file_path+"1-5.csv")
df2 = pd.read_csv(file_path+"6-10.csv")
df3 = pd.read_csv(file_path+"11-15.csv")
df4 = pd.read_csv(file_path+"16-20.csv")
df5 = pd.read_csv(file_path+"21-23,25-26.csv")
df6 = pd.read_csv(file_path+"27-28.csv")

df_nonLanded = pd.concat([df1,df2,df3,df4,df5,df6], axis=0, join='inner')
df_nonLanded = df_nonLanded.drop(["S/N"],axis=1)
df_nonLanded = df_nonLanded.drop(["Building/Project Name"],axis=1)
df_nonLanded = df_nonLanded.drop(["Street Name"],axis=1)
df_nonLanded = df_nonLanded.drop(["No. of Bedroom(for Non-Landed Only)"],axis=1)

df_nonLanded = df_nonLanded.replace({'Jan':'1-1'},regex=True)
df_nonLanded = df_nonLanded.replace({'Feb':'1-2'},regex=True)
df_nonLanded = df_nonLanded.replace({'Mar':'1-3'},regex=True)
df_nonLanded = df_nonLanded.replace({'Apr':'1-4'},regex=True)
df_nonLanded = df_nonLanded.replace({'May':'1-5'},regex=True)
df_nonLanded = df_nonLanded.replace({'Jun':'1-6'},regex=True)
df_nonLanded = df_nonLanded.replace({'Jul':'1-7'},regex=True)
df_nonLanded = df_nonLanded.replace({'Aug':'1-8'},regex=True)
df_nonLanded = df_nonLanded.replace({'Sep':'1-9'},regex=True)
df_nonLanded = df_nonLanded.replace({'Oct':'1-10'},regex=True)
df_nonLanded = df_nonLanded.replace({'Nov':'1-11'},regex=True)
df_nonLanded = df_nonLanded.replace({'Dec':'1-12'},regex=True)

df_nonLanded = df_nonLanded.replace({'-':'/'},regex=True)

#df_nonLanded.to_csv(file_path+"Non-Landed.csv")

#df_nonLanded.head(50)


In [10]:
#pre-set file path
file_path2 = "../dataset/rental-prediction/Landed Properties/"

#combining all into one file for landed
df1 = pd.read_csv(file_path2+"1-5.csv")
df2 = pd.read_csv(file_path2+"6-10.csv")
df3 = pd.read_csv(file_path2+"11-15.csv")
df4 = pd.read_csv(file_path2+"16-20.csv")
df5 = pd.read_csv(file_path2+"21-23,25-26.csv")
df6 = pd.read_csv(file_path2+"27-28.csv")

df_Landed = pd.concat([df1,df2,df3,df4,df5,df6], axis=0, join='inner')
df_Landed = df_Landed.drop(["S/N"],axis=1)
df_Landed = df_Landed.drop(["Building/Project Name"],axis=1)
df_Landed = df_Landed.drop(["Street Name"],axis=1)
df_Landed = df_Landed.drop(["No. of Bedroom(for Non-Landed Only)"],axis=1)

df_Landed = df_Landed.replace({'Jan':'1-1'},regex=True)
df_Landed = df_Landed.replace({'Feb':'1-2'},regex=True)
df_Landed = df_Landed.replace({'Mar':'1-3'},regex=True)
df_Landed = df_Landed.replace({'Apr':'1-4'},regex=True)
df_Landed = df_Landed.replace({'May':'1-5'},regex=True)
df_Landed = df_Landed.replace({'Jun':'1-6'},regex=True)
df_Landed = df_Landed.replace({'Jul':'1-7'},regex=True)
df_Landed = df_Landed.replace({'Aug':'1-8'},regex=True)
df_Landed = df_Landed.replace({'Sep':'1-9'},regex=True)
df_Landed = df_Landed.replace({'Oct':'1-10'},regex=True)
df_Landed = df_Landed.replace({'Nov':'1-11'},regex=True)
df_Landed = df_Landed.replace({'Dec':'1-12'},regex=True)

df_Landed = df_Landed.replace({'-':'/'},regex=True)

#df_Landed.to_csv(file_path2+"Landed.csv")

#dropped the no of bedrooms as there was no data recorded


In [11]:
file_path3 = "../dataset/rental-prediction/Executive Condominiums/"

df1 = pd.read_csv(file_path3+"16-20.csv")
df2 = pd.read_csv(file_path3+"21-23,25-26.csv")
df3 = pd.read_csv(file_path3+"27-28.csv")

df_EC = pd.concat([df1,df2,df3], axis=0, join='inner')
df_EC = df_EC.drop(["S/N"],axis=1)
df_EC = df_EC.drop(["Building/Project Name"],axis=1)
df_EC = df_EC.drop(["Street Name"],axis=1)
df_EC = df_EC.drop(["No. of Bedroom(for Non-Landed Only)"],axis=1)

df_EC = df_EC.replace({'Jan':'1-1'},regex=True)
df_EC = df_EC.replace({'Feb':'1-2'},regex=True)
df_EC = df_EC.replace({'Mar':'1-3'},regex=True)
df_EC = df_EC.replace({'Apr':'1-4'},regex=True)
df_EC = df_EC.replace({'May':'1-5'},regex=True)
df_EC = df_EC.replace({'Jun':'1-6'},regex=True)
df_EC = df_EC.replace({'Jul':'1-7'},regex=True)
df_EC = df_EC.replace({'Aug':'1-8'},regex=True)
df_EC = df_EC.replace({'Sep':'1-9'},regex=True)
df_EC = df_EC.replace({'Oct':'1-10'},regex=True)
df_EC = df_EC.replace({'Nov':'1-11'},regex=True)
df_EC = df_EC.replace({'Dec':'1-12'},regex=True)

df_EC = df_EC.replace({'-':'/'},regex=True)

#df_EC.to_csv(file_path3+"Executive Condominiums.csv")

In [12]:
df = pd.concat([df_nonLanded,df_Landed,df_EC], axis=0, join='inner')

In [13]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 447228 entries, 0 to 405
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Postal District          447228 non-null  int64 
 1   Type                     447228 non-null  object
 2   Monthly Gross Rent($)    447228 non-null  int64 
 3   Floor Area (sq ft)       447228 non-null  object
 4   Lease Commencement Date  447228 non-null  object
dtypes: int64(2), object(3)
memory usage: 20.5+ MB


In [14]:
df.head()

,Postal District,Type,Monthly Gross Rent($),Floor Area (sq ft),Lease Commencement Date
0,4,Non/landed Properties,2650,400 to 500,1/4/21
1,4,Non/landed Properties,7500,1900 to 2000,1/4/21
2,3,Non/landed Properties,5000,1000 to 1100,1/4/21
3,2,Non/landed Properties,5700,800 to 900,1/4/21
4,1,Non/landed Properties,4350,2200 to 2300,1/4/21


In [15]:
now = pd.to_datetime("now")

# Convert month column to 'month' type
df['Lease Commencement Date'] = pd.to_datetime(df['Lease Commencement Date'])
df['Lease Commencement Date'] = (now - df['Lease Commencement Date']).dt.days
# Convert month to ordinal type
#df_nonLanded['Lease Commencement Date'] = df_nonLanded['Lease Commencement Date'].map(datetime.datetime.toordinal)


# Check data types of columns
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 447228 entries, 0 to 405
Data columns (total 5 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   Postal District          447228 non-null  int64 
 1   Type                     447228 non-null  object
 2   Monthly Gross Rent($)    447228 non-null  int64 
 3   Floor Area (sq ft)       447228 non-null  object
 4   Lease Commencement Date  447228 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 20.5+ MB


In [16]:
# Find out mean, median, standard deviation, etc
df.describe()

,Postal District,Monthly Gross Rent($),Lease Commencement Date
count,447228.000000,447228.000000,447228.000000
mean,13.374116,3798.413268,1098.550520
std,6.437851,2576.042626,534.250061
min,1.000000,307.000000,164.000000
25%,9.000000,2400.000000,532.000000
50%,14.000000,3000.000000,1252.000000
75%,19.000000,4250.000000,1620.000000
max,28.000000,170000.000000,1990.000000


In [17]:
# Checks how many rows/columns are there
df.shape

(447228, 5)

## Check for null/empty values

In [18]:
df.keys()

Index(['Postal District', 'Type', 'Monthly Gross Rent($)',
       'Floor Area (sq ft)', 'Lease Commencement Date'],
      dtype='object')

In [19]:
# Remove month, storey_range column later
df = pd.get_dummies(df, columns=['Type', 'Floor Area (sq ft)'])
df.head()



,Postal District,Monthly Gross Rent($),Lease Commencement Date,Type_Detached House,Type_Executive Condominium,Type_Non/landed Properties,Type_Semi/Detached House,Type_Terrace House,Floor Area (sq ft)_1000 to 1100,Floor Area (sq ft)_1000 to 1500,...,Floor Area (sq ft)_6500 to 7000,Floor Area (sq ft)_700 to 800,Floor Area (sq ft)_7000 to 7500,Floor Area (sq ft)_7500 to 8000,Floor Area (sq ft)_800 to 900,Floor Area (sq ft)_900 to 1000,Floor Area (sq ft)_<= 1000,Floor Area (sq ft)_<= 200,Floor Area (sq ft)_>3000,Floor Area (sq ft)_>8000
0,4,2650,164,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,7500,164,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,5000,164,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2,5700,164,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,1,4350,164,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df.shape

(447228, 54)

# Training Model - Regression
- We will test the training of model first.

In [21]:
X = df.drop(['Monthly Gross Rent($)'], axis=1)
Y = df['Monthly Gross Rent($)']

#print(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42,shuffle=True)

X_validation, X_test, y_validation, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [22]:
reg = LinearRegression()
reg.fit(X_train,y_train)
test = reg.predict(X_test)
print(test)

[7074.7941728  4729.62771461 2805.22471185 ... 2769.07312011 3045.56325673
 3686.74393906]


In [27]:
mse = mean_absolute_error(y_train, reg.predict(X_train))
print("Mean Absolute Error on training data: {:.4f}".format(mse))

mse = mean_absolute_error(y_validation, reg.predict(X_validation))
print("Mean Absolute Error on validation data: {:.4f}".format(mse))

Mean Absolute Error on training data: 789.5920
Mean Absolute Error on validation data: 783.2267


In [24]:
#helps to improve accuracy of the model by handling the decresing gradient
clf = ensemble.GradientBoostingRegressor(n_estimators=400, max_depth=5, min_samples_split=2, learning_rate=0.1, loss='ls')

In [25]:
clf.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=5, n_estimators=400)

In [26]:
clf.score(X_test,y_test)

0.7651675615928204